In [2]:
import pandas as pd
df = pd.read_csv('hello_comments/spam_result/new_v0_self_training_results.csv')
print(len(df))

df_filtered = df[df['predicted_label'] == 'Not Spam']
df_filtered.to_csv('hello_comments/for_bert/video_0_filtered_spam.csv', index=False)

1948


In [3]:
print(len(df_filtered))

759


In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd

/Users/wangyuching/Desktop/論文準備/Hello_Comment/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
i = 35
tag_data = pd.read_csv(f"hello_comments/spam_tag/v{i}_tagged.csv")
print(f"tag_data: {len(tag_data)}")

df0 = pd.read_csv(f'hello_comments/spam_tag/video_{i}_ckip_spam_tag.csv')
print(f"df0: {len(df0)}")

# 如果comment_text, published_at, author_name都一樣的話，把tag_data的spam_tag欄位合併到df中
df = df0.merge(
    tag_data[['comment_text', 'published_at', 'author_name', 'spam_tag']],
    on=['comment_text', 'published_at', 'author_name'],
    how='left'
)
df['spam_tag'] = df['spam_tag_y'].combine_first(df['spam_tag_x'])
df = df.drop(columns=['spam_tag_x', 'spam_tag_y'])

print(f"df: {len(df)}")

tag_data: 30
df0: 1503
df: 1503


In [4]:
# 準備標籤和留言
label_list = []
comments = df['cleaned_text'].tolist()

for idx, row in df.iterrows():
    if row['spam_tag'] == "spam":
        label_list.append(1)
    elif row['spam_tag'] == "non-spam":
        label_list.append(0)
    else:
        label_list.append(-1)

# 對應標記：1 = spam，0 = non-spam，-1 = 無標記
labels = np.array(label_list)
comments = np.array(comments)

print(f"總留言數: {len(comments)}")
print(f"有標記的spam數量: {np.sum(labels == 1)}")
print(f"有標記的non-spam數量: {np.sum(labels == 0)}")
print(f"無標記數量: {np.sum(labels == -1)}")

總留言數: 1503
有標記的spam數量: 156
有標記的non-spam數量: 25
無標記數量: 1322


In [5]:
# 拆分成有標籤與無標籤
labeled_mask = labels != -1
unlabeled_mask = labels == -1


In [ ]:
# unlabeled_mask

array([ True,  True,  True, ...,  True,  True,  True], shape=(1948,))

In [6]:
# # 載入 BERT 編碼器
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(comments)

X_labeled = embeddings[labeled_mask]
y_labeled = labels[labeled_mask]

X_unlabeled = embeddings[unlabeled_mask]
unlabeled_comments = comments[unlabeled_mask]

In [7]:
### 測試切分train和test資料集
from sklearn.model_selection import train_test_split

# 切分 train/test
X_train, X_test, y_train, y_test = train_test_split(X_labeled, y_labeled, test_size=0.3, random_state=42)

# 初始模型訓練&評估
clf = LogisticRegression(class_weight='balanced', random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("\n=== 初始模型在測試集的表現 ===")
print(classification_report(y_test, y_pred, digits=3))
print("混淆矩陣：")
print(confusion_matrix(y_test, y_pred))


=== 初始模型在測試集的表現 ===
              precision    recall  f1-score   support

           0      0.889     0.889     0.889         9
           1      0.978     0.978     0.978        46

    accuracy                          0.964        55
   macro avg      0.934     0.934     0.934        55
weighted avg      0.964     0.964     0.964        55

混淆矩陣：
[[ 8  1]
 [ 1 45]]


In [ ]:
# # 初始模型訓練
# # clf = LogisticRegression(random_state=42)
# clf = LogisticRegression(class_weight='balanced', random_state=42)
# clf.fit(X_labeled, y_labeled)

LogisticRegression(class_weight='balanced', random_state=42)

In [8]:
# 對無標籤資料做預測
probs = clf.predict_proba(X_unlabeled)
confidence = np.max(probs, axis=1)
pseudo_labels = clf.predict(X_unlabeled)

In [9]:
## threshold的設定會影響到pseudo label的品質！
threshold = 0.85
selected = confidence >= threshold

print(f"\n=== 加入偽標籤的留言 (信心度 >= {threshold}) ===")
print(f"高信心度預測數量: {np.sum(selected)}")
for i, (cmt, conf, pred) in enumerate(zip(unlabeled_comments[selected], confidence[selected], pseudo_labels[selected])):
    if i < 20:  # 只顯示前10個例子
        print(f"{cmt[:50]}... → {'Spam' if pred==1 else 'Not Spam'} ({conf:.3f})")

# 建立新訓練集（加上偽標籤）
X_new_train = np.concatenate([X_labeled, X_unlabeled[selected]])
y_new_train = np.concatenate([y_labeled, pseudo_labels[selected]])

print(f"\n原始訓練集大小: {len(X_labeled)}")
print(f"加入偽標籤後訓練集大小: {len(X_new_train)}")

# 重新訓練模型
clf_final = LogisticRegression(random_state=42)
clf_final.fit(X_new_train, y_new_train)


=== 加入偽標籤的留言 (信心度 >= 0.85) ===
高信心度預測數量: 26
八年前 我投給了阿北 看見白紙底下有一層綠 四年前我又投給阿北 那層阻礙阿北的綠不見了 只剩下白 那... → Not Spam (0.872)
剥皮猴 猴友疑 3亿探长 你就把陽明山文化大學你的套房103間捐出來 侯友宜真正性格是不適宜當總統 ... → Not Spam (0.864)
我不知道這邊綠營的支持者多不多 有的話記得去看彭文正 陳致曉 夏學理 賀德芬 張凱鈞教授們的遭遇或說... → Not Spam (0.864)
為什麼不讓她自己介紹自己 為什麼講話不能好好講 ㄧ定要跟苗博雅學... → Not Spam (0.852)
撲馬在講資訊戰 民眾黨只能派花瓶講垃圾話呵呵... → Not Spam (0.850)
越來越喜歡Albee了 以前看節目就知道他除了美也很聰明 反應快 沒想到這種拋梗接球也很好耶 還會彈... → Not Spam (0.853)
其他人謹言慎行都是有所企圖或醬缸 像KP這樣真誠才是對的 所以好老闆應該是生氣時 真誠 地罵你是狗或... → Not Spam (0.852)
不能這麼說 每個人喜好不同 對別人來說是最美的也 沒錯阿... → Not Spam (0.851)
官話就是一直繞圈圈 就不會講錯話 或是躲起來不接受訪問 也不會講錯話... → Not Spam (0.860)
真的 後來都在看他們演戲了 XD 現在又在演哪一齣的 最近要不是側翼在哭礦工豪可憐 還不知道萬里的事... → Not Spam (0.856)
老實說那些合約或疑點啥的我根本不想管 因為這就藍白之間的問題 柯文哲早就知道kmt會出招 而柯也已經... → Not Spam (0.872)
垃圾不分藍綠 不代表藍綠都是垃圾 是不是墨綠不重要 因為墨綠也不一定是垃圾... → Not Spam (0.851)
失言會認錯 而不是硬拗 全黨挺一人 請搜尋 柯文哲嗆 放狗咬人 楊寶禎硬拗可愛... → Not Spam (0.865)
所以柯文哲不是墙头草两边摆 而是经常乱说话 所以被骂了多少次了 XD... → Not Spam (0.862)
講柯文哲失言那段雖然好笑 但也側面反映了我們有一個總統候選人 是有錯會認 認了會改的人 當大家都說你... → N

LogisticRegression(random_state=42)

In [ ]:
# # 對所有留言重新預測
# final_preds = clf_final.predict(embeddings)

# print(f"\n=== 最終預測統計 ===")
# print(f"預測為spam的數量: {np.sum(final_preds == 1)}")
# print(f"預測為not spam的數量: {np.sum(final_preds == 0)}")

# # 將結果加回DataFrame
# df['predicted_spam'] = final_preds
# df['predicted_label'] = ['Spam' if label == 1 else 'Not Spam' for label in final_preds]


=== 最終預測統計 ===
預測為spam的數量: 1175
預測為not spam的數量: 773


In [10]:
#  retrain 完模型後，對原本測試集（X_test）預測
y_pred_final = clf_final.predict(X_test)
print("\n=== 偽標註後的模型在測試集的表現 ===")
print(classification_report(y_test, y_pred_final, digits=3))
print("混淆矩陣：")
print(confusion_matrix(y_test, y_pred_final))


=== 偽標註後的模型在測試集的表現 ===
              precision    recall  f1-score   support

           0      1.000     0.889     0.941         9
           1      0.979     1.000     0.989        46

    accuracy                          0.982        55
   macro avg      0.989     0.944     0.965        55
weighted avg      0.982     0.982     0.981        55

混淆矩陣：
[[ 8  1]
 [ 0 46]]


In [58]:
df.to_csv('hello_comments/spam_result/v0_self_training_results.csv', index=False)

In [12]:
# 顯示一些預測結果範例
print(f"\n=== 預測結果範例 ===")
for i in range(min(10, len(df))):
    original_tag = df.iloc[i]['spam_tag'] if pd.notna(df.iloc[i]['spam_tag']) else '無標記'
    predicted_tag = df.iloc[i]['predicted_label']
    comment = df.iloc[i]['cleaned_text'][:50]
    print(f"{comment}... | 原始: {original_tag} | 預測: {predicted_tag}")


=== 預測結果範例 ===


KeyError: 'predicted_label'

In [5]:
import pandas as pd
import os

# 定義要分析的影片清單
file_list = [0, 1, 7, 14, 21, 22, 25, 31, 35]  # 根據你的實際檔案調整

# 建立結果列表
results = []

for i in file_list:
    try:
        # 讀取每個影片的spam結果檔案
        file_path = f"hello_comments/spam_result/v{i}_self_training_results.csv"
        
        # 檢查檔案是否存在
        if not os.path.exists(file_path):
            print(f"檔案不存在: {file_path}")
            continue
            
        df = pd.read_csv(file_path, encoding='utf-8')
        
        # 取得影片標題
        video_title = df['video_title'].iloc[0] if 'video_title' in df.columns else f"Video_{i}"
        
        # 計算spam和not spam的數量
        spam_count = df[df['predicted_label'] == "Spam"].shape[0]
        not_spam_count = df[df['predicted_label'] == "Not Spam"].shape[0]
        total_comment_count = len(df)
        
        # 計算spam比例
        spam_ratio = spam_count / total_comment_count if total_comment_count > 0 else 0
        
        # 將結果加入列表
        results.append({
            'video_id': i,
            'video_title': video_title,
            'spam_count': spam_count,
            'not_spam_count': not_spam_count,
            'total_comments': total_comment_count,
            'spam_ratio': round(spam_ratio, 4)
        })
        
        print(f"影片 {i}: {video_title} - Spam: {spam_count}, Not Spam: {not_spam_count}, 比例: {spam_ratio:.4f}")
        
    except Exception as e:
        print(f"處理影片 {i} 時發生錯誤: {str(e)}")

# 建立DataFrame
spam_summary_df = pd.DataFrame(results)

# 顯示結果
print("\n=== 最終統計結果 ===")
print(spam_summary_df)

# 儲存結果
spam_summary_df.to_csv('hello_comments/spam_result/spam_summary_all_videos.csv', index=False)
print(f"\n結果已儲存至: hello_comments/spam_result/spam_summary_all_videos.csv")

# 顯示整體統計
print(f"\n=== 整體統計 ===")
print(f"總影片數: {len(spam_summary_df)}")
print(f"總留言數: {spam_summary_df['total_comments'].sum()}")
print(f"總spam數: {spam_summary_df['spam_count'].sum()}")
print(f"總not spam數: {spam_summary_df['not_spam_count'].sum()}")
print(f"整體spam比例: {spam_summary_df['spam_count'].sum() / spam_summary_df['total_comments'].sum():.4f}")
print(f"平均spam比例: {spam_summary_df['spam_ratio'].mean():.4f}")

影片 0: 【#賀瓏夜夜秀】1/20 新聞亂報 EP9｜落選夜夜秀謝票大會 - Spam: 1103, Not Spam: 845, 比例: 0.5662
影片 1: 【#賀瓏夜夜秀】1/27 新聞亂報 EP10｜新聞自報 - Spam: 3129, Not Spam: 10141, 比例: 0.2358
影片 7: 【#賀瓏夜夜秀】12/16 新聞亂報 EP6｜Blue不錄了 - Spam: 1620, Not Spam: 515, 比例: 0.7588
影片 14: 【#賀瓏夜夜秀】吳欣盈 參選副總統其實有點Shock - Spam: 2546, Not Spam: 8246, 比例: 0.2359
影片 21: 【#賀瓏夜夜秀】欸！投開票 - Spam: 1307, Not Spam: 3797, 比例: 0.2561
影片 22: 【#賀瓏夜夜秀】欸！特偵組 - Spam: 1959, Not Spam: 2748, 比例: 0.4162
影片 25: 【#賀瓏夜夜秀】欸！蔣公銅像 - Spam: 1747, Not Spam: 2825, 比例: 0.3821
影片 31: 【#賀瓏夜夜秀】趙少康 戰鬥藍的老大另有其人 - Spam: 3824, Not Spam: 5546, 比例: 0.4081
影片 35: 【#賀瓏夜夜秀】黃瀞瑩 偶爾會覺得失言其實也蠻到位 - Spam: 829, Not Spam: 674, 比例: 0.5516

=== 最終統計結果 ===
   video_id                      video_title  spam_count  not_spam_count  \
0         0  【#賀瓏夜夜秀】1/20 新聞亂報 EP9｜落選夜夜秀謝票大會        1103             845   
1         1      【#賀瓏夜夜秀】1/27 新聞亂報 EP10｜新聞自報        3129           10141   
2         7   【#賀瓏夜夜秀】12/16 新聞亂報 EP6｜Blue不錄了        1620             515   
3        14       【#賀瓏夜夜秀】吳欣盈 參選副總統其實有點Shock

In [ ]:
### 如果有建議人工標助的留言，取出後標記再重新抓做成一個v{i}_tagged.csv檔案
import pandas as pd

i = 0

# 建議人工標助的留言
tobe_labeled = pd.read_csv('hello_comments/spam_tag/v0_to_be_labeled_round2.csv')
print(">>> 一共有",len(tobe_labeled), "筆留言建議要人工重新標注")

df_tagged = pd.read_csv(f'hello_comments/pseudo_labeling/v{i}_tagged.csv')

print(">>> 原本的標註檔案有", len(df_tagged), "筆")

# 處理 tobe_labeled 的格式，確保有正確的欄位
if 'comment' in tobe_labeled.columns and 'cleaned_text' not in tobe_labeled.columns:
    # 如果有 comment 欄位但沒有 cleaned_text，就將 comment 重新命名為 cleaned_text
    tobe_labeled['cleaned_text'] = tobe_labeled['comment']

# 確保有 video_id 欄位
if 'video_id' not in tobe_labeled.columns:
    tobe_labeled['video_id'] = f'v{i}'

# 確保有 spam_tag 欄位，如果沒有就設為空白（等待人工標記）
if 'spam_tag' not in tobe_labeled.columns:
    tobe_labeled['spam_tag'] = ''

# 只保留需要的三個欄位
tobe_labeled_formatted = tobe_labeled[['video_id', 'cleaned_text', 'spam_tag']].copy()

# 將建議人工標助的留言加入到原本的標註檔案中
df_tagged = pd.concat([df_tagged, tobe_labeled_formatted], ignore_index=True)

# 確保最終結果只有三個欄位：video_id, cleaned_text, spam_tag
df_tagged = df_tagged[['video_id', 'cleaned_text', 'spam_tag']].copy()

print(">>> 加入建議人工標助的留言後，標註檔案有", len(df_tagged), "筆")

# 儲存為適合上傳 Google Spreadsheet 的格式
df_tagged.to_csv(f'hello_comments/pseudo_labeling/v{i}_relabel.csv', index=False)
print(f">>> 已儲存為 hello_comments/pseudo_labeling/v{i}_relabel.csv")

# 顯示前幾筆資料確認格式
print("\n>>> 前5筆資料預覽：")
print(df_tagged.head())

>>> 一共有 20 筆留言建議要人工重新標注
>>> 原本的標註檔案有 100 筆
>>> 加入建議人工標助的留言後，標註檔案有 120 筆
>>> 已儲存為 hello_comments/pseudo_labeling/v0_relabel.csv

>>> 前5筆資料預覽：
  video_id                                       cleaned_text  spam_tag
0       v0  假設馬 造謠 郭有涉及伊案 那就看郭有沒有提告馬 如果有提告 這是ㄧ件案子 馬有沒有軍情洩密...  non-spam
1       v0                                          鄭ㄧˋ 忘了 德欸      spam
2       v0    請問香港人了解華人社會嗎 我們華人 台灣人 是不會跟老闆說東西不好吃的 不好吃只能摸摸鼻子離開  non-spam
3       v0                                        阿頭算是少數正向獲利的  non-spam
4       v0                  年轻人会去看吗 台视和Tvbs的播放量和评论量连夜夜秀的零头都不到  non-spam
